# Assignment (Lesson 05)
# Data Preparation and Feature Selection
Steps in a data science project
1. Acquire data
2. Exploratory Data analysis (EDA)
3. Data Processing
    1. Data Preparation
    2. Feature Selection
4. Predictive Analytics

### Import Packages
Python, like most programming languages, has pre-made software methods.  These pre-made software methods are organized and combined by topic into packages.  The packages that we want are:
- numpy (numerical python)
- pandas (panel data aka tables)
- sklearn (sci-kit learn for predictive analytics)
- matplotlib (data plotting for matrix-like data)  

We need to "import" these packages so that we can use their methods in our code.

In [44]:
# import packages
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
# Allow inline plotting in Jupyter Notebook
%matplotlib inline

## Data Preparation on the Mammographic Masses Dataset (Mamm)
### Acquire data
We will get our data from the University of California, Irvine Machine Learning Repository.  Our dataset was used to determine the effectivity of radiological evaluations of breast cancer diagnoses in women who have breast tumors.  You can get some information on the data from here:  http://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.names

In [45]:
# csv file:
url = "../data/mammographic_masses.data"
# Alternate data source:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.data"

# Download the data
Mamm = pd.read_csv(url, header=None)

# Replace the default column names (0, 1, 2, 3, 4, 5) with meaningful names
Mamm.columns = ["BI_RADS", "Age", "Shape", "Margin", "Density", "Severity"]

Mamm.head()

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1


### Some preliminary EDA:
"BI_RADS", and "Density" are ordinal columns.  We will assume that they are numeric.  
"Age" and "Severity" are numeric columns.    
"Shape" and "Margin" are category columns but they are encoded as integers.  

Show the actual data types of these columns.  Can you guess why the data types of these 5 columns are `object`?

<span style="color:red" float:right>[0 point]</span>

In [46]:
Mamm.shape

(961, 6)

In [47]:
# Add code here
Mamm.dtypes

BI_RADS     object
Age         object
Shape       object
Margin      object
Density     object
Severity     int64
dtype: object

> There's probably NaNs or bad data in the columns that are `object`s

### Some Data Processing
In the following sections you will do the following to the Mamm dataframe:
- Replace unusable entries with null/nan  
- Change types of data.
- Correct unexpected values (outliers)
- decode category data    
- Consolidate categories in category data 

#### Replace Missing Values with Nulls
Coerce all columns, even category columns, that contain missing values to numeric data using `pd.to_numeric`.  You might get an error, like `Unable to parse string`.  You need to tell `pd.to_numeric` that it should **coerce** the casting when it encounters a value that it cannot parse.  The category columns in this dataset are encoded as integers.  We will make use of that encoding.  Any non-numeric value will be replaced with a nan and you will get nans for missing numeric and category values.  After you replace all the non-numeric values, present the first five rows with `Mamm.head()`.

<span style="color:red" float:right>[1 point]</span>

In [48]:
# Coerce all the data to numeric data
# Coercion will introduce nans/nulls for the non-numeric values in all columns
# Because the categories are encoded as integers, the missing categories will also be nans/nulls after coercion.
# Add code here
for col in Mamm.dtypes.items():
    if col[1] == 'object':
        Mamm[col[0]] = pd.to_numeric(Mamm[col[0]],errors='coerce')

In [49]:
Mamm

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1
...,...,...,...,...,...,...
956,4.0,47.0,2.0,1.0,3.0,0
957,4.0,56.0,4.0,5.0,3.0,1
958,4.0,64.0,4.0,5.0,3.0,0
959,5.0,66.0,4.0,5.0,3.0,1


In [50]:
Mamm.dtypes

BI_RADS     float64
Age         float64
Shape       float64
Margin      float64
Density     float64
Severity      int64
dtype: object

#### Replace Outliers
Values that are obviously incorrect are often replaced with averages.  Often, outlier replacements with averages are inappropriate because the extreme values have some meaning.  For instance, from the data dictionary we know that BI_RADS should range from 1 to 5.  BI_RADS values beyond 1 and 5 were added by physicians who did not adhere to the accepted range.  In this case, BI_RADS greater than 5 should be "clipped" at 5 and BI_RADS less than 1 should be "clipped" at 1. 

<span style="color:red" float:right>[1 point]</span>

In [51]:
# Cap BI_RADS values to a range of 1 to 5
# Add code here
Mamm.loc[Mamm['BI_RADS'] > 5, 'BI_RADS'] = 5
Mamm.loc[Mamm['BI_RADS'] < 1, 'BI_RADS'] = 1

### Consolidate and decode category columns

Decoding a category is when categories are coded as numbers and we replace those numbers with actual categories.  
Consolidating (aka binning or grouping) of categories is means that multiple categories are renamed to a single category.  
The decoding and consolidating of categories can occur at the same time.  

- Shape
  - The original category codes are: round=1; oval=2; lobular=3; irregular=4;  
  - The proper consolidated category decoding is: 1 $\rightarrow$ oval; 2 $\rightarrow$ oval; 3 $\rightarrow$ lobular; 4 $\rightarrow$ irregular;  
- Margin
  - The orginal category codes are: circumscribed=1; microlobulated=2; obscured=3; ill-defined=4; spiculated=5  
  - The proper consolidated category decodes are: 1 $\rightarrow$ circumscribed; 2 $\rightarrow$ ill-defined; 3 $\rightarrow$ ill-defined; 4 $\rightarrow$ ill-defined; 5 $\rightarrow$ spiculated;

After you decode and consolidate, present the first five rows with `Mamm.head()`. 

<span style="color:red" float:right>[1 point]</span>

In [52]:
# The category columns are decoded and categories are consolidated

# The Shape variable is decoded as follows:  1 and 2 to oval;  3 to lobular; 4 to irregular
# Add code here
mapper = {1:"oval", 2:"oval", 3:"lobular", 4:"irregular"}
Mamm.replace({'Shape':mapper}, inplace=True)
# The Margin variable is decoded as follows:  1 to circumscribed;  2, 3, 4 to ill_defined; 5 to spiculated
# Add code here
mapper = {1:"circumscribed",2:"ill-defined",3:"ill-defined",4:"ill-defined",5:"spiculated"}
Mamm.replace({'Margin':mapper}, inplace=True)

#####
#> Converting to categories for later
Mamm[['Shape','Margin']] = Mamm[['Shape','Margin']].astype('category')

# Present the first few rows
# Add code here
Mamm

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,lobular,spiculated,3.0,1
1,4.0,43.0,oval,circumscribed,NaN,1
2,5.0,58.0,irregular,spiculated,3.0,1
3,4.0,28.0,oval,circumscribed,3.0,0
4,5.0,74.0,oval,spiculated,NaN,1
...,...,...,...,...,...,...
956,4.0,47.0,oval,circumscribed,3.0,0
957,4.0,56.0,irregular,spiculated,3.0,1
958,4.0,64.0,irregular,spiculated,3.0,0
959,5.0,66.0,irregular,spiculated,3.0,1


> I choose to show the first 5 & last 5 via the call/print method in a notebook so I can more readily detect any drift or oddities I create. Not very applicabable here but is useful to me when working with the larger datasets at work.

In [53]:
Mamm.dtypes

BI_RADS      float64
Age          float64
Shape       category
Margin      category
Density      float64
Severity       int64
dtype: object

### Some More EDA
- Show the shape of the dataframe
- Use the `pandas` `isna` method to show the distribution of nulls among the columns.
  
<span style="color:red" float:right>[0 point]</span>

In [54]:
# Show the shape of the data frame
# Add code here
Mamm.shape

(961, 6)

In [55]:
# Show the distribution of nulls among the columns
# Add code here
Mamm.isna().describe()

,BI_RADS,Age,Shape,Margin,Density,Severity
count,961,961,961,961,961,961
unique,2,2,2,2,2,1
top,False,False,False,False,False,False
freq,959,956,930,913,885,961


### Drop Rows with Multiple Missing Values
When a row has too many missing values, then it should not be used.  We can stipulate a threshold requirement of available values per row.  We will require that each row contains at least 5 values.  This requirement means that no row is allowed more than 1 missing value.  
Remove the rows that have more than one missing value.  
- Use the `pandas` `dropna` method and set the `thresh` argument.  
- Show the shape of the dataframe after you drop the rows with multiple nulls. 
- Use the `pandas` `isna` method to show the number of nulls per column after dropping rows with multiple nulls

<span style="color:red" float:right>[1 point]</span>

In [56]:
# Drop rows
# Add code here
Mamm.dropna(thresh=5, inplace=True)

# Show the shape of the data frame
# Add code here
Mamm.shape

(931, 6)

In [57]:
# Show the distribution of nulls among the columns
# Add code here
Mamm.isna().describe()

,BI_RADS,Age,Shape,Margin,Density,Severity
count,931,931,931,931,931,931
unique,2,2,2,2,2,1
top,False,False,False,False,False,False
freq,930,926,914,909,875,931


## Impute Missing Values
Use the median values to impute missing values for true numerical columns (`Age`, `BI_RADS`, `Density`).  `Margin` and `Shape` originally looked numeric, but they are categorical.  Therefore, do not use median on `Margin` and `Shape`.  

### Determine the imputation values for Age

In [58]:
for col in Mamm.columns:
    if pd.api.types.is_numeric_dtype(Mamm[col]):
        print((f"The column is {col} with medians\n\tpd.Seris.median() : {Mamm[col].median()} | numpy.nanmediam : {np.nanmedian(Mamm[col].values)}"))


The column is BI_RADS with medians
	pd.Seris.median() : 4.0 | numpy.nanmediam : 4.0
The column is Age with medians
	pd.Seris.median() : 57.0 | numpy.nanmediam : 57.0
The column is Density with medians
	pd.Seris.median() : 3.0 | numpy.nanmediam : 3.0
The column is Severity with medians
	pd.Seris.median() : 0.0 | numpy.nanmediam : 0.0


> It appears, based on this dataset, that the `pandas.Series.median()` and `numpy.nanmediam()` methods are effectively equal. I'll lean on Pnadas only from here.

> Also, I'll loop over the dataframe rather than explicityly call out columns.

```python
# Replace missing age values with the median 
MedianAge = np.nanmedian(Mamm.loc[:,"Age"])
HasNanAge = pd.isnull(Mamm.loc[:,"Age"])
print('Now we replace', HasNanAge.sum(),'missing age values with the age median (', MedianAge, ')')
Mamm.loc[HasNanAge, "Age"] = MedianAge
Mamm.isna().sum(axis=0)
```

### Impute Missing values for BI_RADS and Density
Assign the column medians to the null values in the respective numeric columns.
- Use the `pandas` `isnull` method to identify the nulls
- Use the `numpy` `nanmedian` to determine the median for imputation
- Use the `pandas` `isna` method to show the number of nulls per column after the imputation   
  
<span style="color:red" float:right>[1 point]</span>

In [59]:
Mamm.isna().describe()

,BI_RADS,Age,Shape,Margin,Density,Severity
count,931,931,931,931,931,931
unique,2,2,2,2,2,1
top,False,False,False,False,False,False
freq,930,926,914,909,875,931


In [60]:
for col in Mamm.columns:
    if pd.api.types.is_numeric_dtype(Mamm[col]):
        Mamm.loc[pd.isna(Mamm[col]), col] = Mamm[col].median()

In [61]:
Mamm.isna().describe()

,BI_RADS,Age,Shape,Margin,Density,Severity
count,931,931,931,931,931,931
unique,1,1,2,2,1,1
top,False,False,False,False,False,False
freq,931,931,914,909,931,931


### Replace missing values for the two categorical columns
- Use `pandas` `value_counts()` method to determine the distribution of categories in `Shape` and `Margin` before imputation.
- Use `pandas` `isnull()` method to identify the missing values
- Assign the most common value to the null values in the respective categorical columns. 
- After the imputation, use the `pandas` `isna` method to show the number of nulls after the imputation.
- Use `pandas` `value_counts()` method to determine the distribution of categories after imputation.

<span style="color:red" float:right>[1 point]</span>

In [62]:
Mamm['Shape'].value_counts(dropna=False), Mamm['Margin'].value_counts(dropna=False)

(Shape
 oval         422
 irregular    399
 lobular       93
 NaN           17
 Name: count, dtype: int64,
 Margin
 ill-defined      417
 circumscribed    357
 spiculated       135
 NaN               22
 Name: count, dtype: int64)

In [63]:
for col in Mamm.dtypes.items():
    if col[1] == 'category':
        Mamm.loc[Mamm[col[0]].isna(),col[0]] = Mamm[col[0]].mode().values[0]

In [64]:
Mamm['Shape'].value_counts(dropna=False), Mamm['Margin'].value_counts(dropna=False)

(Shape
 oval         439
 irregular    399
 lobular       93
 Name: count, dtype: int64,
 Margin
 ill-defined      439
 circumscribed    357
 spiculated       135
 Name: count, dtype: int64)

### One hot encode the categorical variables
- Use `OneHotEncoder` from `sklearn.preprocessing` to one-hot encode the two categorical variables, `Shape` and `Margin`.
- Make sure that the new columns have descriptive hybrid names by using the `get_feature_names_out` method.
- Add the new binary columns to the dataframe.
- drop the original columns, `Shape` and `Margin`
- Show the first few rows of the dataframe.

<span style="color:red" float:right>[3 point]</span>

> Review what the dataframe currently looks like.

In [65]:
Mamm

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,lobular,spiculated,3.0,1
1,4.0,43.0,oval,circumscribed,3.0,1
2,5.0,58.0,irregular,spiculated,3.0,1
3,4.0,28.0,oval,circumscribed,3.0,0
4,5.0,74.0,oval,spiculated,3.0,1
...,...,...,...,...,...,...
956,4.0,47.0,oval,circumscribed,3.0,0
957,4.0,56.0,irregular,spiculated,3.0,1
958,4.0,64.0,irregular,spiculated,3.0,0
959,5.0,66.0,irregular,spiculated,3.0,1


> Start the OneHotEncoding process

In [66]:
from sklearn.preprocessing import OneHotEncoder

> Ok... package is in. Now to create the object in memory, encode the category columns (Shape, Margin), add those to the dataframe, drop the original columns, and show the tranformation process. Fun!

In [67]:
onehot = OneHotEncoder(sparse_output=False)
encode_cols = Mamm.select_dtypes('category').copy()
onehot.fit(encode_cols)
print(f"The new column names are: {onehot.get_feature_names_out()}")

The new column names are: ['Shape_irregular' 'Shape_lobular' 'Shape_oval' 'Margin_circumscribed'
 'Margin_ill-defined' 'Margin_spiculated']


In [68]:
Mamm[onehot.get_feature_names_out()] = onehot.transform(encode_cols)

In [69]:
Mamm

,BI_RADS,Age,Shape,Margin,Density,Severity,Shape_irregular,Shape_lobular,Shape_oval,Margin_circumscribed,Margin_ill-defined,Margin_spiculated
0,5.0,67.0,lobular,spiculated,3.0,1,0.0,1.0,0.0,0.0,0.0,1.0
1,4.0,43.0,oval,circumscribed,3.0,1,0.0,0.0,1.0,1.0,0.0,0.0
2,5.0,58.0,irregular,spiculated,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0
3,4.0,28.0,oval,circumscribed,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0
4,5.0,74.0,oval,spiculated,3.0,1,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
956,4.0,47.0,oval,circumscribed,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0
957,4.0,56.0,irregular,spiculated,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0
958,4.0,64.0,irregular,spiculated,3.0,0,1.0,0.0,0.0,0.0,0.0,1.0
959,5.0,66.0,irregular,spiculated,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0


In [70]:
Mamm = Mamm.drop(['Shape','Margin'], axis=1)

In [71]:
Mamm

,BI_RADS,Age,Density,Severity,Shape_irregular,Shape_lobular,Shape_oval,Margin_circumscribed,Margin_ill-defined,Margin_spiculated
0,5.0,67.0,3.0,1,0.0,1.0,0.0,0.0,0.0,1.0
1,4.0,43.0,3.0,1,0.0,0.0,1.0,1.0,0.0,0.0
2,5.0,58.0,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0
3,4.0,28.0,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0
4,5.0,74.0,3.0,1,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
956,4.0,47.0,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0
957,4.0,56.0,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0
958,4.0,64.0,3.0,0,1.0,0.0,0.0,0.0,0.0,1.0
959,5.0,66.0,3.0,1,1.0,0.0,0.0,0.0,0.0,1.0


## End of Data Preparation on the Mammographic Masses Dataset (Mamm)



## Feature Selection on the Indian  Liver Patient Dataset (ILPD)
Feature selection is a process of removing features that are redundant and that could lead to overfitting, singular matrices, and other problems associated with high cardinality (Curse of dimensionality:  https://en.wikipedia.org/wiki/Curse_of_dimensionality)

### Acquire Data

We will get our data from the University of California, Irvine Machine Learning Repository. Our dataset was used to determine if blood test data could be sufficient to identify liver disease in rural areas with few physicians.

In [72]:
# csv file:
url = "../data/Indian Liver Patient Dataset (ILPD).csv"
# Alternate data source:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00225/Indian Liver Patient Dataset (ILPD).csv"
url = url.replace(" ", "%20")

# Download the data
ILPD = pd.read_csv(url, header=None)

# Replace the default column names (0, 1, 2, 3, 4, 5) with meaningful names
ILPD.columns = ["Age","Gender","DB","TB","Alkphos","Sgpt","Sgot","TPr","ALB","AGRatio","Selector"]

ILPD

,Age,Gender,DB,TB,Alkphos,Sgpt,Sgot,TPr,ALB,AGRatio,Selector
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


### Data Preparation for ILPD
- All columns should be numeric and continuous
    - Remove binary columns (numeric and categorical) because their mutual information scores will be lower
    - Remove any categorical columns
- Remove or impute any missing values

<span style="color:red" float:right>[1 point]</span>

In [73]:
ILPD

,Age,Gender,DB,TB,Alkphos,Sgpt,Sgot,TPr,ALB,AGRatio,Selector
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


> I'll loop over the entire dataframe and run a test against the quantity of unique values to drop the columns

In [74]:
for col in ILPD.columns:
    if ILPD[col].astype('category').describe()['unique'] == 2:
        ILPD = ILPD.drop(col, axis=1)

In [75]:
ILPD

,Age,DB,TB,Alkphos,Sgpt,Sgot,TPr,ALB,AGRatio
0,65,0.7,0.1,187,16,18,6.8,3.3,0.90
1,62,10.9,5.5,699,64,100,7.5,3.2,0.74
2,62,7.3,4.1,490,60,68,7.0,3.3,0.89
3,58,1.0,0.4,182,14,20,6.8,3.4,1.00
4,72,3.9,2.0,195,27,59,7.3,2.4,0.40
...,...,...,...,...,...,...,...,...,...
578,60,0.5,0.1,500,20,34,5.9,1.6,0.37
579,40,0.6,0.1,98,35,31,6.0,3.2,1.10
580,52,0.8,0.2,245,48,49,6.4,3.2,1.00
581,31,1.3,0.5,184,29,32,6.8,3.4,1.00


> It appears that only the AGRatio column contans NaNs, so I'll impute that column

In [76]:
ILPD.isna().any()

Age        False
DB         False
TB         False
Alkphos    False
Sgpt       False
Sgot       False
TPr        False
ALB        False
AGRatio     True
dtype: bool

In [77]:
# Impute values or remove rows with nulls
# Add Code here
for col in ILPD.columns:
    if pd.api.types.is_numeric_dtype(ILPD[col]) and ILPD[col].isna().any():
        ILPD.loc[pd.isna(ILPD[col]), col] = ILPD[col].median()


In [78]:
ILPD.isna().any()

Age        False
DB         False
TB         False
Alkphos    False
Sgpt       False
Sgot       False
TPr        False
ALB        False
AGRatio    False
dtype: bool

In [79]:
ILPD.describe()

,Age,DB,TB,Alkphos,Sgpt,Sgot,TPr,ALB,AGRatio
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.946947
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318495
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.930000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000


### Mutual Information
https://en.wikipedia.org/wiki/Mutual_information
Below is a wrapper for determining the mutual information between two continuous (numeric) variables

In [80]:
from sklearn.metrics import mutual_info_score
# x is the first input variable
# y is the second input variable
# bins is the number of discretized values that will be used for the two input variables
def calc_MI(x, y, bins=80):
    if (bins > 1):
        c_xy = np.histogram2d(x, y, bins)[0]
        mi = mutual_info_score(None, None, contingency = c_xy)
    else:
        mi = mutual_info_score(x, y)
    return mi

### Create method to list  all column pairs together with their mutual information score
Write a method called `listMutualInformationScores`.  It uses the above method (`calc_MI`) in a loop to find the mutual information between all possible pairs of coulmns in the data.  The input to the function is the dataframe of continuous variables, specifically the prepared ILPD dataset.  

The method returns a list of lists.  Each inner list contains three items:  the x-column, the y-column, and the mutual information score. The list of lists contains every possible pair of coulmns in the data.  The result should have a form similar to the following, except that the outer list is much longer and contains all possible column pairs:  
`[['Alkphos', 'Sgot', 0.33],
['Sgot', 'AGRatio', 0.23],
['Age', 'Sgot', 0.35],
['Sgpt', 'AGRatio', 0.30],
['Sgot', 'ALB', 0.29],
['Sgot', 'TPr', 0.33]]`

<span style="color:red" float:right>[3 point]</span>

In [81]:
# define the method listMutualInformationScores

def listMutualInformationScores(df):
    "Something about the function"
    
    output = []
    for n,col in enumerate(df.columns):
        for c in df.columns[n:]:
            if col == c:
                continue
            else:
                output.append((col, c, calc_MI(ILPD[col],ILPD[c])))

    return output

In [82]:
# Run the method listMutualInformationScores
listMutualInformationScores(ILPD)

[('Age', 'DB', 0.561414195138507),
 ('Age', 'TB', 0.746054371711554),
 ('Age', 'Alkphos', 0.9151537816562115),
 ('Age', 'Sgpt', 0.4777319788430886),
 ('Age', 'Sgot', 0.3483804487141835),
 ('Age', 'TPr', 1.4560887594744552),
 ('Age', 'ALB', 1.239784790243598),
 ('Age', 'AGRatio', 0.9702392168614418),
 ('DB', 'TB', 1.4009679188357755),
 ('DB', 'Alkphos', 0.515720778828546),
 ('DB', 'Sgpt', 0.40506677478144626),
 ('DB', 'Sgot', 0.35062567364592356),
 ('DB', 'TPr', 0.5390915524796547),
 ('DB', 'ALB', 0.47169756336375573),
 ('DB', 'AGRatio', 0.44428984777444414),
 ('TB', 'Alkphos', 0.6317155237041214),
 ('TB', 'Sgpt', 0.4713132801033194),
 ('TB', 'Sgot', 0.41504245969685455),
 ('TB', 'TPr', 0.7411186413121742),
 ('TB', 'ALB', 0.6246399613112071),
 ('TB', 'AGRatio', 0.5788670719004747),
 ('Alkphos', 'Sgpt', 0.4137551620237777),
 ('Alkphos', 'Sgot', 0.32593918702234126),
 ('Alkphos', 'TPr', 0.8109652265789737),
 ('Alkphos', 'ALB', 0.727161368142061),
 ('Alkphos', 'AGRatio', 0.5824022145370717

### Present the mutual information results
- Package the output into a dataframe
- Sort the rows in descending order of mutual information
- Present the dataframe

The first column could be x, the second column could be called y and the third column could be called mi.  x and y are the pair of columns pair and mi is the pair's mutual information score.  The result should have a form similar to the following:

| x | y | mi |
| --- | --- | --- |
| Age | Sgot | 0.35 |
| Sgot | TPr | 0.33 |
| Alkphos | Sgot | 0.33 |
| Sgpt | AGRatio | 0.30 |
| Sgot | ALB | 0.29 |
| Sgot | AGRatio | 0.23 |

<span style="color:red" float:right>[1 point]</span>

In [83]:
# Present the results as dataframe
ILPD_mutual = pd.DataFrame(listMutualInformationScores(ILPD), columns=['x','y','mi'])
ILPD_mutual

,x,y,mi
0,Age,DB,0.561414
1,Age,TB,0.746054
2,Age,Alkphos,0.915154
3,Age,Sgpt,0.477732
4,Age,Sgot,0.348380
5,Age,TPr,1.456089
6,Age,ALB,1.239785
7,Age,AGRatio,0.970239
8,DB,TB,1.400968
9,DB,Alkphos,0.515721


### Discussion on Mutual Information in ILPD
Lets assume a threshold of 1 for the mutual information score

Which columns would you eliminate? Why?  To answer these questions, you may need to read-up on feature selection with mutual information score.

<span style="color:red" float:right>[1 point]</span>

> In reading a little about what Mutual Information (or Information Gain) is from https://machinelearningmastery.com/information-gain-and-mutual-information/, I believe I can say the following:
>
> Setting the threshold at 1 is saying that any score above one and those to columns/variables are sufficiently dependent to use going forward but the columns below 1 are too independent and thus too unpredictable to continue into the construction of a model. 
>
> Based on this idea and the code below, that leaves ['Alkphos', 'Sgpt', 'Sgot'] as independent variables that are probably too unpredictable to be effectively modeled.

In [84]:
ILPD.columns

Index(['Age', 'DB', 'TB', 'Alkphos', 'Sgpt', 'Sgot', 'TPr', 'ALB', 'AGRatio'], dtype='object')

In [85]:
ILPD_mutual[ILPD_mutual['mi'] > 1]

,x,y,mi
5,Age,TPr,1.456089
6,Age,ALB,1.239785
8,DB,TB,1.400968
33,TPr,ALB,1.372006
35,ALB,AGRatio,1.002026


In [86]:
feature_cols = []
for row in ILPD_mutual[ILPD_mutual['mi'] > 1].iterrows():
    if row[1]['x'] not in feature_cols:
        feature_cols.append(row[1]['x'])
    if row[1]['y'] not in feature_cols:
        feature_cols.append(row[1]['y'])
        
not_features = [col for col in ILPD.columns.to_list() if col not in feature_cols]


feature_cols, not_features

(['Age', 'TPr', 'ALB', 'DB', 'TB', 'AGRatio'], ['Alkphos', 'Sgpt', 'Sgot'])